# Get Started with Flower Framework

Welcome to Federated Learning Tutorial using Flower Framework. Flower is a unified approach to federated learning, analytics, and evaluation. Open source, python, and easy to learn and personalize.

https://flower.ai/

## Step 0: Preparation

Before we begin with any actual code, let's make sure that we have everything we need.

### Instaling dependencies
First, we should install the necessary packages

In [ ]:
# Linux
!pip install protobuf==4.25.3
!pip install -q flwr[simulation] matplotlib
#!pip install --upgrade tensorflow-metadata

# MacOs
#!pip3 install -U 'flwr[simulation]' torch torchvision scipy

Now that we have all dependencies installed, we can import everything we need for this tutorial:

In [ ]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import requests
import pandas as pd
import numpy as np

import flwr as fl
from flwr.common import Metrics

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

It is possible to switch to a runtime that has GPU acceleration enabled (on Google Colab: Runtime > Change runtime type > Hardware accelerator: GPU > Save). Note, however, that Google Colab is not always able to offer GPU acceleration. If you see an error related to GPU availability in one of the following sections, consider switching back to CPU-based execution by setting DEVICE = torch.device("cpu"). If the runtime has GPU acceleration enabled, you should see the output Training on cuda, otherwise it'll say Training on cpu.

### Loading the data

Federated learning can be applied to many different types of tasks across different domains. In this tutorial, we introduce federated learning by training a simple Linear Regression on the popular Abalone dataset. Abalone can be used in classification and regression tasks using 9 features: Sex, Length, Diameter, Height, Whole_weight, Shucked_weight, Viscera_weight, Shell_weight, and Rings.

We simulate having multiple datasets from multiple organizations (also called the "cross-silo" setting in federated learning) by splitting the original Abalone dataset into multiple partitions. Each partition will represent the data from a single organization. We're doing this purely for experimentation purposes.

Each organization will act as a client in the federated learning system. So having 3 organizations participate in a federation means having 3 clients connected to the federated learning server.


Let's now create the Federated Dataset abstraction that from flwr-datasets that partitions the Abalone. We will create small training and test set for each edge device and wrap each of them into a PyTorch DataLoader:

In [ ]:
NUM_CLIENTS = 2

def load_data():
    data = load_iris()
    X, y = data.data, data.target

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

def partition_data(X, y, num_clients):
    partition_size = len(X) // num_clients
    partitions = [(X[i * partition_size:(i + 1) * partition_size], y[i * partition_size:(i + 1) * partition_size])
                  for i in range(num_clients)]
    return partitions


### Implementing Flower Client

In [ ]:
from collections import OrderedDict
from typing import Dict, List, Tuple
from sklearn.metrics import log_loss, accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score

from flwr.common import NDArrays, Scalar, Status, Code

# in the python file you want to add custom messages to the Flower log
from logging import INFO, DEBUG
from flwr.common.logger import log

class SklearnClient(fl.client.NumPyClient):
    def __init__(self, X_train, y_train, X_test, y_test, cid):
      #super().__init__()
      self.model = LogisticRegression(penalty="l2", max_iter=100, warm_start=True)
      self.X_train = X_train
      self.y_train = y_train
      self.X_test = X_test
      self.y_test = y_test
      self.cid = cid
      self.set_initial_params()

    def set_initial_params(self):
        """Sets initial parameters as zeros Required since model params are uninitialized
        until model.fit is called.

        But server asks for initial parameters from clients at launch. Refer to
        sklearn.linear_model.LogisticRegression documentation for more information.
        """
        unique_classes = np.unique(self.y_train)
        n_classes = len(unique_classes) # Iris has 3 classes
        n_features = self.X_train.shape[1] # Number of features in dataset
        self.model.classes_ = np.array([i for i in range(n_classes)])

        self.model.coef_ = np.zeros((n_classes, n_features))
        if self.model.fit_intercept:
            self.model.intercept_ = np.zeros((n_classes,))

    def get_parameters(self, config=None):
      if not hasattr(self.model, "coef_"):
        self.model.fit(self.X_train[:1], self.y_train[:1])

      if self.model.fit_intercept:
        params = [
            self.model.coef_,
            self.model.intercept_
        ]
      else:
        params = [
            self.model.coef_
        ]
      return [param.astype(np.float32) for param in params]

    def set_parameters(self, parameters):
      if self.model.fit_intercept:
        self.model.coef_ = parameters[0]
        self.model.intercept_ = parameters[1]
      else:
        self.model.coef_ = parameters[0]
      return self.model

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.fit(self.X_train, self.y_train)
        # print(f"Training finished for round {config}")
        return self.get_parameters(), len(self.X_train), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        y_pred = self.model.predict(self.X_test)
        loss = 1 - accuracy_score(self.y_test, y_pred)
        # loss = log_loss(self.y_test, model.predict_proba(X_test))
        accuracy = self.model.score(self.X_test, self.y_test)
        cid_ = self.cid
        report = classification_report(self.y_test, y_pred)

        # print("******* accuracy ")
        # print(accuracy)
        # print("*** Classification Report ***:")
        # print(classification_report(self.y_test, y_pred))

        message = "Client " + str(cid_) + " accuracy: " + str(accuracy)

        log(DEBUG, message)

        #log(DEBUG, f"Client {self.cid} is doing fit() with config:")

        return loss, len(self.X_test), {"accuracy": accuracy}


### Using the Virtual Client Engine

In [ ]:
def client_fn(cid: str) -> fl.client.Client:
  X_train, X_test, y_train, y_test = load_data()
  num_clients = NUM_CLIENTS  # Number of clients should match the number of partitions
  partitions = partition_data(X_train, y_train, num_clients)

  partition_id = int(cid)

  # print("##### Partition id #######" + cid)

  X_train_cid, y_train_cid = partitions[partition_id]

  #return SklearnClient(X_train_cid, y_train_cid, X_test, y_test).to_client()
  return SklearnClient(X_train_cid, y_train_cid, X_test, y_test, cid)

### Start the training

In [ ]:
def fit_config(server_round) -> Dict:
    """Send round number to client."""
    config = {
        "server_round": server_round
    }
    return config

# Define the strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=1.0,
    min_fit_clients=1,
    min_evaluate_clients=1,
    min_available_clients=1,
    on_fit_config_fn=fit_config
)

# Simulation configuration
client_resources = {"num_cpus": 1}
num_clients = NUM_CLIENTS
num_rounds = 5


In [ ]:
fl.simulation.start_simulation(
 # strategy=strategy, # the strategy that will construct a client
  client_fn=client_fn, # a function to construct a client
  num_clients=num_clients, # total number of clients in the experiment
  config=fl.server.ServerConfig(num_rounds=2), #let's run for 1 rounds
  client_resources=client_resources,
)


In [ ]:

## Salvar os resultados de cada cliente em um ficheiro csv
## Exibir o resultado de cada cliente no último bloco de código
